# Encoder / Decoder models

Encoding and decoding is a framework / approach rather than being a machine learning model. It usually includes two models, one encoding and the other one decoding. 

Especially useful for anomaly detection and denoising. 

The idea is you take a complex data, and try to represent it with as few as possible features. Then from these features you could re-trace your steps and recreate the original data but without the noise. 

Here, we will try to find the **latent** space for the triangles using the generated images. Ideally, we should be able to represent each with only 4 numbers; `a,b,c,` and an `angle`. Then we could use a decoder to either; recreate a new triangle image, or given 3 of the parameters, estimate the remaining one.

In [1]:

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import os, sys

# Add the project root to sys.path (one level up from this notebook)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from hypotai.data import generate_triangle_data
from hypotai.plotting import plot_triangle

2025-06-23 17:00:10.258128: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-23 17:00:10.299070: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-23 17:00:10.350237: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750690810.391913   47885 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750690810.404487   47885 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750690810.485170   47885 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

Here we use a CNN to decode the triangles into a latent space.

In [ ]:
# Encoder
encoder = models.Sequential([
    layers.Input(shape=(64, 64, 1)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(4, name="latent_vector")  # 🧠 bottleneck
])

# Decoder (optional if reconstructing image)
decoder = models.Sequential([
    layers.Input(shape=(4,)),
    layers.Dense(8*8*64, activation='relu'),
    layers.Reshape((8, 8, 64)),
    layers.Conv2DTranspose(64, (3, 3), strides=2, padding='same', activation='relu'),
    layers.Conv2DTranspose(32, (3, 3), strides=2, padding='same', activation='relu'),
    layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')  # Reconstructed image
])
